# Mô hình LSI (Latent Semantic Indexing)

Notebook này triển khai mô hình LSI hoàn chỉnh theo yêu cầu:

1. **Giới thiệu mô hình**: Phương pháp biểu diễn tài liệu và truy vấn, nguyên tắc tính toán độ liên quan
2. **Chọn term**: Phương pháp xác định term với ví dụ minh họa
3. **Công thức tính trọng số term**: TF-IDF và các thành phần
4. **Lập chỉ mục**: Cấu trúc chỉ mục và quá trình xử lý tài liệu
5. **Xử lý truy vấn**: Phân tích truy vấn và tính toán độ tương đồng
6. **Đánh giá mô hình**: Đánh giá trên ngữ liệu Cranfield theo P, R và MAP nội suy 11 điểm TREC


In [ ]:
# Import thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import os
import glob
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data if needed
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

print("Đã import thành công các thư viện cần thiết!")


In [ ]:
# Load dữ liệu Cranfield
def load_cranfield_documents():
    """Đọc tài liệu từ bộ dữ liệu Cranfield"""
    documents = {}
    doc_path = "../dataset/Crandfield/Cranfield/"
    
    if not os.path.exists(doc_path):
        doc_path = "dataset/Crandfield/Cranfield/"
    
    txt_files = glob.glob(os.path.join(doc_path, "*.txt"))
    
    for file_path in txt_files:
        doc_id = os.path.basename(file_path).replace('.txt', '')
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read().strip()
                if content:
                    documents[doc_id] = content
        except Exception as e:
            print(f"Lỗi khi đọc file {file_path}: {e}")
    
    return documents

def load_cranfield_queries():
    """Đọc truy vấn từ bộ dữ liệu Cranfield"""
    queries = {}
    query_paths = ["../dataset/Crandfield/query.txt", "dataset/Crandfield/query.txt"]
    
    for query_path in query_paths:
        if os.path.exists(query_path):
            break
    else:
        print("Không tìm thấy file query.txt")
        return queries
    
    try:
        with open(query_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) >= 2:
                    query_id = parts[0]
                    query_text = parts[1]
                    queries[query_id] = query_text
    except Exception as e:
        print(f"Lỗi khi đọc queries: {e}")
    
    return queries

# Load dữ liệu
documents = load_cranfield_documents()
queries = load_cranfield_queries()

print(f"Đã load {len(documents)} tài liệu")
print(f"Đã load {len(queries)} truy vấn")

if documents:
    first_doc_id = list(documents.keys())[0]
    print(f"\nVí dụ tài liệu đầu tiên (ID: {first_doc_id}):")
    print(f"{documents[first_doc_id][:200]}...")

if queries:
    first_query_id = list(queries.keys())[0]
    print(f"\nVí dụ truy vấn đầu tiên (ID: {first_query_id}):")
    print(f"{queries[first_query_id]}")


In [ ]:
# Hàm tiền xử lý văn bản
def preprocess_text(text):
    """
    Tiền xử lý văn bản: tokenization, loại bỏ stopwords, stemming
    """
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    
    # Chuyển về chữ thường và tokenize
    text = text.lower()
    tokens = word_tokenize(text)
    
    # Loại bỏ dấu câu, số, và từ quá ngắn
    tokens = [token for token in tokens if token.isalpha() and len(token) > 2]
    
    # Loại bỏ stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming
    tokens = [stemmer.stem(token) for token in tokens]
    
    return tokens

# Ví dụ minh họa quá trình chọn term
if documents:
    sample_text = list(documents.values())[0][:100]  # Lấy 100 ký tự đầu
else:
    sample_text = "experimental investigation of the aerodynamics of a wing in a slipstream"

print("=== VÍ DỤ MINH HỌA QUÁ TRÌNH CHỌN TERM ===")
print(f"Văn bản gốc:\n'{sample_text}'")
print(f"\nDộ dài: {len(sample_text)} ký tự\n")

# Bước 1: Tokenization và chuyển về chữ thường
tokens = word_tokenize(sample_text.lower())
print(f"Bước 1 - Tokenization:")
print(f"Tokens: {tokens}")
print(f"Số lượng tokens: {len(tokens)}\n")

# Bước 2: Lọc chỉ giữ lại các từ (loại bỏ dấu câu, số)
alpha_tokens = [token for token in tokens if token.isalpha()]
print(f"Bước 2 - Lọc từ (chỉ giữ ký tự chữ cái):")
print(f"Tokens: {alpha_tokens}")
print(f"Số lượng: {len(alpha_tokens)}\n")

# Bước 3: Loại bỏ từ quá ngắn (< 3 ký tự)
long_tokens = [token for token in alpha_tokens if len(token) > 2]
print(f"Bước 3 - Loại bỏ từ ngắn (< 3 ký tự):")
print(f"Tokens: {long_tokens}")
print(f"Số lượng: {len(long_tokens)}\n")

# Bước 4: Loại bỏ stopwords
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in long_tokens if token not in stop_words]
print(f"Bước 4 - Loại bỏ stopwords:")
print(f"Stopwords được loại: {[token for token in long_tokens if token in stop_words]}")
print(f"Tokens còn lại: {filtered_tokens}")
print(f"Số lượng: {len(filtered_tokens)}\n")

# Bước 5: Stemming
stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
print(f"Bước 5 - Stemming:")
print("Từ gốc -> Từ sau stemming:")
for original, stemmed in zip(filtered_tokens, stemmed_tokens):
    if original != stemmed:
        print(f"  {original} -> {stemmed}")
print(f"\nTerms cuối cùng: {stemmed_tokens}")
print(f"Số lượng terms: {len(stemmed_tokens)}")

# Phân tích thống kê trên toàn bộ collection
print(f"\n=== PHÂN TÍCH THỐNG KÊ TRÊN TOÀN BỘ COLLECTION ===")

if documents:
    all_terms = []
    doc_count = 0
    for doc_id, content in list(documents.items())[:100]:  # Phân tích 100 tài liệu đầu
        terms = preprocess_text(content)
        all_terms.extend(terms)
        doc_count += 1
    
    term_counter = Counter(all_terms)
    vocabulary_size = len(term_counter)
    total_terms = len(all_terms)
    
    print(f"Số tài liệu đã phân tích: {doc_count}")
    print(f"Tổng số terms: {total_terms:,}")
    print(f"Kích thước từ vựng (unique terms): {vocabulary_size:,}")
    print(f"Tỷ lệ unique terms: {vocabulary_size/total_terms:.2%}")
    
    print(f"\n10 terms xuất hiện nhiều nhất:")
    for term, count in term_counter.most_common(10):
        print(f"  '{term}': {count} lần")
    
    print(f"\n10 terms xuất hiện ít nhất:")
    rare_terms = [item for item in term_counter.most_common()[-10:]]
    for term, count in rare_terms:
        print(f"  '{term}': {count} lần")
else:
    print("Không có dữ liệu để phân tích")


In [ ]:
# Tạo TF-IDF vectorizer tùy chỉnh
class CustomTfidfVectorizer:
    def __init__(self, max_features=1000, min_df=2, max_df=0.8):
        self.max_features = max_features
        self.min_df = min_df
        self.max_df = max_df
        self.vocabulary_ = {}
        self.idf_ = {}
        
    def fit_transform(self, documents):
        # Tiền xử lý tài liệu
        processed_docs = []
        for doc in documents:
            terms = preprocess_text(doc)
            processed_docs.append(' '.join(terms))
        
        # Sử dụng TfidfVectorizer của sklearn với dữ liệu đã xử lý
        self.vectorizer = TfidfVectorizer(
            max_features=self.max_features,
            min_df=self.min_df,
            max_df=self.max_df,
            lowercase=False,  # Đã chuyển về chữ thường
            stop_words=None,  # Đã loại bỏ stopwords
            token_pattern=r'\b\w+\b'
        )
        
        matrix = self.vectorizer.fit_transform(processed_docs)
        self.vocabulary_ = self.vectorizer.vocabulary_
        self.idf_ = self.vectorizer.idf_
        
        return matrix
    
    def transform(self, documents):
        processed_docs = []
        for doc in documents:
            terms = preprocess_text(doc)
            processed_docs.append(' '.join(terms))
        return self.vectorizer.transform(processed_docs)

# Ví dụ tính toán TF-IDF thủ công
print("=== VÍ DỤ TÍNH TOÁN TF-IDF THỦ CÔNG ===")

if documents:
    # Lấy 3 tài liệu đầu tiên để demo
    sample_docs = list(documents.items())[:3]
    print("Tài liệu mẫu:")
    for doc_id, content in sample_docs:
        print(f"Doc {doc_id}: {content[:80]}...")
    
    # Tiền xử lý các tài liệu
    processed_sample_docs = []
    for doc_id, content in sample_docs:
        terms = preprocess_text(content)
        processed_sample_docs.append(terms)
        print(f"\nDoc {doc_id} - Terms sau xử lý: {terms[:10]}...")
    
    # Tính TF-IDF cho một term cụ thể
    target_term = "wing"  # Chọn term "wing" để demo
    target_term_stemmed = PorterStemmer().stem(target_term)
    
    print(f"\n=== TÍNH TF-IDF CHO TERM: '{target_term}' (stemmed: '{target_term_stemmed}') ===")
    
    # Tính TF cho từng document
    tfs = []
    for i, terms in enumerate(processed_sample_docs):
        tf_raw = terms.count(target_term_stemmed)  # TF thô
        tf_normalized = tf_raw / len(terms) if len(terms) > 0 else 0  # TF chuẩn hóa
        tf_log = 1 + np.log(tf_raw) if tf_raw > 0 else 0  # TF logarithm
        
        tfs.append({
            'doc_id': sample_docs[i][0],
            'tf_raw': tf_raw,
            'tf_normalized': tf_normalized,
            'tf_log': tf_log,
            'doc_length': len(terms)
        })
        
        print(f"Doc {sample_docs[i][0]}:")
        print(f"  Số lần xuất hiện: {tf_raw}")
        print(f"  Độ dài document: {len(terms)} terms")
        print(f"  TF thô: {tf_raw}")
        print(f"  TF chuẩn hóa: {tf_normalized:.4f}")
        print(f"  TF logarithm: {tf_log:.4f}")
    
    # Tính IDF
    N = len(processed_sample_docs)  # Tổng số documents
    df = sum(1 for terms in processed_sample_docs if target_term_stemmed in terms)  # Document frequency
    
    idf_basic = np.log(N / df) if df > 0 else 0
    idf_smooth = np.log(N / (1 + df)) + 1
    
    print(f"\n=== TÍNH IDF ===")
    print(f"Tổng số documents (N): {N}")
    print(f"Số documents chứa term '{target_term_stemmed}' (df): {df}")
    print(f"IDF cơ bản: log({N}/{df}) = {idf_basic:.4f}")
    print(f"IDF smoothed: log({N}/{1+df}) + 1 = {idf_smooth:.4f}")
    
    # Tính TF-IDF cuối cùng
    print(f"\n=== TF-IDF CUỐI CÙNG ===")
    print("Doc ID | TF thô | TF norm | TF log | IDF    | TF-IDF(norm) | TF-IDF(log)")
    print("-------|--------|---------|--------|--------|--------------|------------")
    
    for tf_data in tfs:
        tfidf_norm = tf_data['tf_normalized'] * idf_basic
        tfidf_log = tf_data['tf_log'] * idf_basic
        print(f"{tf_data['doc_id']:6} | {tf_data['tf_raw']:6} | {tf_data['tf_normalized']:7.4f} | {tf_data['tf_log']:6.4f} | {idf_basic:6.4f} | {tfidf_norm:12.4f} | {tfidf_log:11.4f}")

# Tạo ma trận TF-IDF cho toàn bộ collection
if documents:
    print(f"\n=== TẠO MA TRẬN TF-IDF CHO TOÀN BỘ COLLECTION ===")
    
    doc_list = list(documents.values())
    doc_ids = list(documents.keys())
    
    # Khởi tạo TF-IDF vectorizer
    tfidf_vectorizer = CustomTfidfVectorizer(max_features=500, min_df=2, max_df=0.8)
    
    # Tạo ma trận TF-IDF
    tfidf_matrix = tfidf_vectorizer.fit_transform(doc_list)
    
    print(f"Kích thước ma trận TF-IDF: {tfidf_matrix.shape}")
    print(f"Số tài liệu: {tfidf_matrix.shape[0]}")
    print(f"Số features (terms): {tfidf_matrix.shape[1]}")
    print(f"Density (tỷ lệ non-zero): {tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]):.4f}")
    
    # Hiển thị một số thống kê
    feature_names = tfidf_vectorizer.vectorizer.get_feature_names_out()
    print(f"\nMột số terms trong vocabulary: {feature_names[:20]}")
    
    # Hiển thị TF-IDF values cho tài liệu đầu tiên
    first_doc_tfidf = tfidf_matrix[0].toarray().flatten()
    non_zero_indices = np.nonzero(first_doc_tfidf)[0]
    
    print(f"\nTài liệu đầu tiên - Top 10 terms với TF-IDF cao nhất:")
    sorted_indices = non_zero_indices[np.argsort(first_doc_tfidf[non_zero_indices])[::-1]]
    for i, idx in enumerate(sorted_indices[:10]):
        term = feature_names[idx]
        tfidf_val = first_doc_tfidf[idx]
        print(f"  {i+1:2d}. '{term}': {tfidf_val:.4f}")
else:
    print("Không có dữ liệu để tạo ma trận TF-IDF")


In [ ]:
# Xây dựng chỉ mục LSI
class LSIModel:
    def __init__(self, n_components=100):
        self.n_components = n_components
        self.svd = TruncatedSVD(n_components=n_components, random_state=42)
        self.tfidf_vectorizer = None
        self.doc_ids = None
        self.lsi_matrix = None
        
    def fit(self, documents, doc_ids):
        """
        Xây dựng chỉ mục LSI từ collection tài liệu
        """
        self.doc_ids = doc_ids
        
        print("Bước 1: Tạo ma trận TF-IDF...")
        self.tfidf_vectorizer = CustomTfidfVectorizer(max_features=500, min_df=2, max_df=0.8)
        tfidf_matrix = self.tfidf_vectorizer.fit_transform(documents)
        
        print(f"Ma trận TF-IDF gốc: {tfidf_matrix.shape}")
        print(f"Density: {tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]):.4f}")
        
        print("Bước 2: Áp dụng SVD...")
        self.lsi_matrix = self.svd.fit_transform(tfidf_matrix)
        
        print(f"Ma trận LSI sau SVD: {self.lsi_matrix.shape}")
        print(f"Explained variance ratio (10 thành phần đầu): {self.svd.explained_variance_ratio_[:10]}")
        print(f"Tổng explained variance: {sum(self.svd.explained_variance_ratio_):.4f}")
        
        return self
    
    def transform_query(self, query):
        """Chuyển đổi truy vấn sang không gian LSI"""
        query_tfidf = self.tfidf_vectorizer.transform([query])
        query_lsi = self.svd.transform(query_tfidf)
        return query_lsi
    
    def search(self, query, top_k=10):
        """Tìm kiếm tài liệu tương tự"""
        query_lsi = self.transform_query(query)
        similarities = cosine_similarity(query_lsi, self.lsi_matrix).flatten()
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in top_indices:
            results.append({
                'doc_id': self.doc_ids[idx],
                'similarity': similarities[idx],
                'rank': len(results) + 1
            })
        
        return results

# Xây dựng mô hình LSI
if documents and len(documents) > 0:
    print("=== XÂY DỰNG CHỈ MỤC LSI ===")
    
    doc_list = list(documents.values())
    doc_ids = list(documents.keys())
    
    # Tạo mô hình LSI
    lsi_model = LSIModel(n_components=50)  # Sử dụng 50 components
    lsi_model.fit(doc_list, doc_ids)
    
    # Phân tích SVD components
    print(f"\n=== PHÂN TÍCH CÁC THÀNH PHẦN SVD ===")
    print(f"Số singular values: {len(lsi_model.svd.singular_values_)}")
    print(f"10 singular values lớn nhất: {lsi_model.svd.singular_values_[:10]}")
    
    # Vẽ biểu đồ explained variance
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(range(1, min(21, len(lsi_model.svd.explained_variance_ratio_) + 1)), 
             lsi_model.svd.explained_variance_ratio_[:20], 'bo-')
    plt.title('Explained Variance Ratio theo Component')
    plt.xlabel('Component')
    plt.ylabel('Explained Variance Ratio')
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    cumulative_variance = np.cumsum(lsi_model.svd.explained_variance_ratio_)
    plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'ro-')
    plt.title('Cumulative Explained Variance')
    plt.xlabel('Số lượng Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nCumulative explained variance với {lsi_model.n_components} components: {cumulative_variance[-1]:.4f}")
    
    # Hiển thị biểu diễn documents trong không gian LSI
    print(f"\n=== BIỂU DIỄN DOCUMENTS TRONG KHÔNG GIAN LSI ===")
    print(f"Document đầu tiên trong không gian LSI (10 chiều đầu):")
    print(lsi_model.lsi_matrix[0][:10])
    print(f"Kích thước biểu diễn cho mỗi document: {lsi_model.lsi_matrix[0].shape}")
    
    # Test chỉ mục với truy vấn mẫu
    sample_query = "aerodynamic wing flow"
    print(f"\n=== TEST CHỈ MỤC VỚI TRUY VẤN MẪU ===")
    print(f"Truy vấn: '{sample_query}'")
    
    results = lsi_model.search(sample_query, top_k=5)
    print("\nTop 5 tài liệu tương tự:")
    for result in results:
        if result['doc_id'] in documents:
            doc_content = documents[result['doc_id']][:100] + "..."
            print(f"Hạng {result['rank']}: Doc {result['doc_id']} (độ tương tự: {result['similarity']:.4f})")
            print(f"Nội dung: {doc_content}")
            print()
    
    # So sánh kích thước lưu trữ
    original_size = lsi_model.tfidf_vectorizer.vectorizer.transform([' '.join(preprocess_text(doc)) for doc in doc_list]).data.nbytes
    lsi_size = lsi_model.lsi_matrix.nbytes
    compression_ratio = original_size / lsi_size if lsi_size > 0 else 0
    
    print(f"=== SO SÁNH KÍCH THƯỚC LUU TRỮ ===")
    print(f"Kích thước ma trận TF-IDF gốc: {original_size / 1024:.2f} KB")
    print(f"Kích thước ma trận LSI: {lsi_size / 1024:.2f} KB")
    print(f"Tỷ lệ nén: {compression_ratio:.2f}x")
    
else:
    print("Không có dữ liệu để xây dựng chỉ mục LSI")


In [ ]:
# Demo chi tiết quá trình xử lý truy vấn
def detailed_query_processing(lsi_model, query, show_steps=True):
    """
    Minh họa chi tiết các bước xử lý truy vấn
    """
    if show_steps:
        print(f"=== XỬ LÝ TRUY VẤN CHI TIẾT ===")
        print(f"Truy vấn gốc: '{query}'")
    
    # Bước 1: Tiền xử lý truy vấn
    preprocessed_terms = preprocess_text(query)
    preprocessed_query = ' '.join(preprocessed_terms)
    
    if show_steps:
        print(f"1. Truy vấn sau tiền xử lý: '{preprocessed_query}'")
        print(f"   Terms: {preprocessed_terms}")
    
    # Bước 2: Chuyển đổi sang TF-IDF vector
    query_tfidf = lsi_model.tfidf_vectorizer.transform([query])
    query_tfidf_dense = query_tfidf.toarray().flatten()
    
    if show_steps:
        print(f"2. TF-IDF vector shape: {query_tfidf.shape}")
        non_zero_indices = np.nonzero(query_tfidf_dense)[0]
        print(f"   Terms có trọng số non-zero: {len(non_zero_indices)}")
        
        if len(non_zero_indices) > 0:
            feature_names = lsi_model.tfidf_vectorizer.vectorizer.get_feature_names_out()
            print("   Top terms với trọng số cao nhất:")
            sorted_indices = non_zero_indices[np.argsort(query_tfidf_dense[non_zero_indices])[::-1]]
            for i, idx in enumerate(sorted_indices[:5]):
                term = feature_names[idx]
                weight = query_tfidf_dense[idx]
                print(f"     {i+1}. '{term}': {weight:.4f}")
    
    # Bước 3: Chuyển đổi sang không gian LSI
    query_lsi = lsi_model.svd.transform(query_tfidf)
    query_lsi_flat = query_lsi.flatten()
    
    if show_steps:
        print(f"3. LSI vector shape: {query_lsi.shape}")
        print(f"   LSI representation (5 chiều đầu): {query_lsi_flat[:5]}")
        print(f"   LSI vector norm: {np.linalg.norm(query_lsi_flat):.4f}")
    
    # Bước 4: Tính độ tương đồng
    similarities = cosine_similarity(query_lsi, lsi_model.lsi_matrix).flatten()
    
    if show_steps:
        print(f"4. Tính độ tương đồng:")
        print(f"   Số documents được so sánh: {len(similarities)}")
        print(f"   Similarity min: {similarities.min():.4f}")
        print(f"   Similarity max: {similarities.max():.4f}")
        print(f"   Similarity trung bình: {similarities.mean():.4f}")
    
    # Bước 5: Xếp hạng
    ranked_indices = np.argsort(similarities)[::-1]
    
    return query_lsi, similarities, ranked_indices

# Test xử lý truy vấn với nhiều queries
if 'lsi_model' in locals() and queries:
    test_queries = list(queries.items())[:3]  # Lấy 3 queries đầu tiên
    
    print("=== DEMO XỬ LÝ TRUY VẤN ===\n")
    
    for i, (query_id, query_text) in enumerate(test_queries):
        print(f"Truy vấn {i+1} (ID: {query_id}): {query_text}")
        query_lsi, similarities, ranked_indices = detailed_query_processing(lsi_model, query_text)
        
        # Hiển thị top 3 kết quả
        print("Top 3 kết quả:")
        for j in range(3):
            if j < len(ranked_indices):
                doc_idx = ranked_indices[j]
                doc_id = lsi_model.doc_ids[doc_idx]
                sim_score = similarities[doc_idx]
                if doc_id in documents:
                    doc_content = documents[doc_id][:80] + "..."
                    print(f"  Hạng {j+1}: Doc {doc_id} (similarity: {sim_score:.4f})")
                    print(f"           {doc_content}")
        print("\n" + "="*60 + "\n")

# So sánh LSI vs TF-IDF baseline
if 'lsi_model' in locals() and documents:
    print("=== SO SÁNH LSI VS TF-IDF BASELINE ===")
    test_query = "aerodynamic wing design"
    print(f"Truy vấn test: '{test_query}'")
    
    # Kết quả từ LSI
    query_lsi, lsi_similarities, lsi_ranked = detailed_query_processing(lsi_model, test_query, show_steps=False)
    
    # Kết quả từ TF-IDF baseline (không SVD)
    query_tfidf = lsi_model.tfidf_vectorizer.transform([test_query])
    doc_tfidf_matrix = lsi_model.tfidf_vectorizer.vectorizer.transform(
        [' '.join(preprocess_text(doc)) for doc in doc_list]
    )
    tfidf_similarities = cosine_similarity(query_tfidf, doc_tfidf_matrix).flatten()
    tfidf_ranked = np.argsort(tfidf_similarities)[::-1]
    
    print("\nSo sánh Top 5 kết quả:")
    print("Hạng | LSI Model              | TF-IDF Baseline")
    print("-----|------------------------|------------------------")
    for i in range(5):
        if i < len(lsi_ranked) and i < len(tfidf_ranked):
            lsi_doc_id = lsi_model.doc_ids[lsi_ranked[i]]
            lsi_sim = lsi_similarities[lsi_ranked[i]]
            
            tfidf_doc_id = lsi_model.doc_ids[tfidf_ranked[i]]
            tfidf_sim = tfidf_similarities[tfidf_ranked[i]]
            
            print(f"{i+1:4d} | Doc {lsi_doc_id:>3} (sim: {lsi_sim:.3f}) | Doc {tfidf_doc_id:>3} (sim: {tfidf_sim:.3f})")
    
    # Tính overlap trong top 10
    top_10_lsi = set(lsi_model.doc_ids[idx] for idx in lsi_ranked[:10])
    top_10_tfidf = set(lsi_model.doc_ids[idx] for idx in tfidf_ranked[:10])
    overlap = len(top_10_lsi.intersection(top_10_tfidf))
    
    print(f"\nDocuments chung trong top 10: {overlap}/10")
    print(f"Điều này cho thấy LSI {'có thể' if overlap < 7 else 'ít'} tìm ra documents khác biệt so với TF-IDF thuần")
    
# Phân tích semantic similarity
if 'lsi_model' in locals():
    print(f"\n=== PHÂN TÍCH SEMANTIC SIMILARITY ===")
    
    # Test với các queries có từ đồng nghĩa
    semantic_queries = [
        "aircraft wing design",
        "airplane wing structure", 
        "plane aerodynamic surface"
    ]
    
    print("Test khả năng tìm kiếm semantic với các queries tương tự:")
    query_vectors = []
    
    for query in semantic_queries:
        query_lsi = lsi_model.transform_query(query)
        query_vectors.append(query_lsi.flatten())
        
        results = lsi_model.search(query, top_k=3)
        print(f"\nQuery: '{query}'")
        print("Top 3 results:")
        for result in results:
            if result['doc_id'] in documents:
                print(f"  Doc {result['doc_id']} (sim: {result['similarity']:.3f})")
    
    # Tính similarity giữa các query vectors
    print(f"\nĐộ tương đồng giữa các queries:")
    for i in range(len(semantic_queries)):
        for j in range(i+1, len(semantic_queries)):
            sim = cosine_similarity([query_vectors[i]], [query_vectors[j]])[0][0]
            print(f"  '{semantic_queries[i]}' vs '{semantic_queries[j]}': {sim:.3f}")
    
else:
    print("Mô hình LSI chưa được khởi tạo. Vui lòng chạy cell trước đó.")


In [ ]:
# Hàm đánh giá mô hình
def calculate_precision_recall(relevant_docs, retrieved_docs, k=None):
    """Tính precision và recall tại cutoff k"""
    if k is not None:
        retrieved_docs = retrieved_docs[:k]
    
    relevant_set = set(relevant_docs)
    retrieved_set = set(retrieved_docs)
    
    relevant_retrieved = relevant_set.intersection(retrieved_set)
    
    precision = len(relevant_retrieved) / len(retrieved_set) if retrieved_set else 0
    recall = len(relevant_retrieved) / len(relevant_set) if relevant_set else 0
    
    return precision, recall

def calculate_average_precision(relevant_docs, retrieved_docs):
    """Tính Average Precision cho một query"""
    if not relevant_docs:
        return 0.0
    
    relevant_set = set(relevant_docs)
    ap = 0.0
    relevant_count = 0
    
    for i, doc_id in enumerate(retrieved_docs):
        if doc_id in relevant_set:
            relevant_count += 1
            precision_at_i = relevant_count / (i + 1)
            ap += precision_at_i
    
    return ap / len(relevant_docs) if relevant_docs else 0.0

def interpolate_precision_recall(precision_recall_pairs):
    """Nội suy precision tại 11 recall levels chuẩn"""
    recall_levels = np.arange(0.0, 1.1, 0.1)
    interpolated_precisions = []
    
    # Sắp xếp theo recall
    precision_recall_pairs.sort(key=lambda x: x[1])
    
    for target_recall in recall_levels:
        # Tìm precision cao nhất tại recall >= target_recall
        max_precision = 0.0
        for precision, recall in precision_recall_pairs:
            if recall >= target_recall:
                max_precision = max(max_precision, precision)
        interpolated_precisions.append(max_precision)
    
    return list(zip(recall_levels, interpolated_precisions))

# Tạo relevance judgments giả lập cho demo
def create_synthetic_relevance_judgments(queries, documents, lsi_model, num_queries=10):
    """
    Tạo relevance judgments giả lập dựa trên TF-IDF similarity
    Trong thực tế, đây sẽ là ground truth từ human assessors
    """
    qrels = {}
    
    query_items = list(queries.items())[:num_queries]
    
    for query_id, query_text in query_items:
        # Sử dụng TF-IDF để tìm documents liên quan (pseudo ground truth)
        query_tfidf = lsi_model.tfidf_vectorizer.transform([query_text])
        doc_tfidf_matrix = lsi_model.tfidf_vectorizer.vectorizer.transform(
            [' '.join(preprocess_text(doc)) for doc in doc_list]
        )
        similarities = cosine_similarity(query_tfidf, doc_tfidf_matrix).flatten()
        
        # Coi top 5% documents có similarity cao nhất là relevant
        threshold = np.percentile(similarities, 95)
        relevant_indices = np.where(similarities >= threshold)[0]
        
        qrels[query_id] = [lsi_model.doc_ids[idx] for idx in relevant_indices]
    
    return qrels

# TF-IDF Baseline Model
class TFIDFModel:
    """Mô hình TF-IDF baseline để so sánh"""
    def __init__(self, tfidf_vectorizer):
        self.tfidf_vectorizer = tfidf_vectorizer
        self.doc_matrix = None
        self.doc_ids = None
        
    def fit(self, documents, doc_ids):
        self.doc_ids = doc_ids
        processed_docs = [' '.join(preprocess_text(doc)) for doc in documents]
        self.doc_matrix = self.tfidf_vectorizer.vectorizer.transform(processed_docs)
        return self
        
    def search(self, query, top_k=10):
        query_vector = self.tfidf_vectorizer.transform([query])
        similarities = cosine_similarity(query_vector, self.doc_matrix).flatten()
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in top_indices:
            results.append({
                'doc_id': self.doc_ids[idx],
                'similarity': similarities[idx],
                'rank': len(results) + 1
            })
        return results

def evaluate_model(model, queries, qrels, model_name, top_k=50):
    """Đánh giá một mô hình sử dụng P, R và MAP"""
    precisions_at_k = []
    recalls_at_k = []
    average_precisions = []
    all_interpolated_points = []
    
    print(f"\nĐang đánh giá {model_name}...")
    
    evaluated_queries = 0
    for query_id, query_text in queries.items():
        if query_id not in qrels:
            continue
            
        relevant_docs = qrels[query_id]
        if not relevant_docs:
            continue
            
        # Lấy kết quả tìm kiếm
        results = model.search(query_text, top_k=top_k)
        retrieved_docs = [r['doc_id'] for r in results]
        
        # Tính các metrics
        precision, recall = calculate_precision_recall(relevant_docs, retrieved_docs, k=top_k)
        ap = calculate_average_precision(relevant_docs, retrieved_docs)
        
        precisions_at_k.append(precision)
        recalls_at_k.append(recall)
        average_precisions.append(ap)
        
        # Tính precision-recall curve để nội suy
        pr_pairs = []
        for i in range(1, min(len(retrieved_docs), 20) + 1):
            p, r = calculate_precision_recall(relevant_docs, retrieved_docs, k=i)
            pr_pairs.append((p, r))
        
        interpolated = interpolate_precision_recall(pr_pairs)
        all_interpolated_points.append(interpolated)
        
        evaluated_queries += 1
    
    # Tính metrics tổng thể
    avg_precision = np.mean(precisions_at_k) if precisions_at_k else 0
    avg_recall = np.mean(recalls_at_k) if recalls_at_k else 0
    map_score = np.mean(average_precisions) if average_precisions else 0
    
    # Tính 11-point interpolated precision
    if all_interpolated_points:
        recall_levels = np.arange(0.0, 1.1, 0.1)
        mean_interpolated_precisions = []
        
        for i, recall_level in enumerate(recall_levels):
            precisions_at_level = [points[i][1] for points in all_interpolated_points]
            mean_interpolated_precisions.append(np.mean(precisions_at_level))
    else:
        mean_interpolated_precisions = [0.0] * 11
    
    return {
        'precision': avg_precision,
        'recall': avg_recall,
        'map': map_score,
        'interpolated_precisions': mean_interpolated_precisions,
        'num_queries': evaluated_queries
    }

# Thực hiện đánh giá
if 'lsi_model' in locals() and documents and queries:
    print("=== ĐÁNH GIÁ MÔ HÌNH TRÊN NGỮ LIỆU CRANFIELD ===")
    
    # Tạo relevance judgments giả lập
    print("Tạo relevance judgments giả lập...")
    qrels = create_synthetic_relevance_judgments(queries, documents, lsi_model, num_queries=15)
    
    print(f"Đã tạo relevance judgments cho {len(qrels)} queries")
    for query_id, relevant_docs in list(qrels.items())[:3]:
        print(f"Query {query_id}: {len(relevant_docs)} relevant documents")
        if query_id in queries:
            print(f"  Text: {queries[query_id][:60]}...")
    
    # Tạo TF-IDF baseline model
    print("\nTạo TF-IDF baseline model...")
    tfidf_baseline = TFIDFModel(lsi_model.tfidf_vectorizer)
    tfidf_baseline.fit(doc_list, doc_ids)
    
    # Đánh giá cả hai mô hình
    lsi_results = evaluate_model(lsi_model, queries, qrels, "LSI Model")
    tfidf_results = evaluate_model(tfidf_baseline, queries, qrels, "TF-IDF Baseline")
    
    # Hiển thị kết quả
    print("\n" + "="*70)
    print("KẾT QUẢ ĐÁNH GIÁ")
    print("="*70)
    
    print(f"\n📊 LSI Model:")
    print(f"  Average Precision@50: {lsi_results['precision']:.4f}")
    print(f"  Average Recall@50:    {lsi_results['recall']:.4f}")
    print(f"  MAP:                  {lsi_results['map']:.4f}")
    print(f"  Số queries đánh giá:  {lsi_results['num_queries']}")
    
    print(f"\n📊 TF-IDF Baseline:")
    print(f"  Average Precision@50: {tfidf_results['precision']:.4f}")
    print(f"  Average Recall@50:    {tfidf_results['recall']:.4f}")
    print(f"  MAP:                  {tfidf_results['map']:.4f}")
    print(f"  Số queries đánh giá:  {tfidf_results['num_queries']}")
    
    # So sánh hiệu suất
    print(f"\n🔍 So sánh hiệu suất:")
    if tfidf_results['map'] > 0:
        map_improvement = ((lsi_results['map'] - tfidf_results['map']) / tfidf_results['map']) * 100
        precision_improvement = ((lsi_results['precision'] - tfidf_results['precision']) / tfidf_results['precision']) * 100
        
        print(f"  MAP improvement:       {map_improvement:+.2f}%")
        print(f"  Precision improvement: {precision_improvement:+.2f}%")
        
        if map_improvement > 0:
            print("  ✅ LSI model có hiệu suất tốt hơn TF-IDF baseline")
        else:
            print("  ⚠️  TF-IDF baseline có hiệu suất tốt hơn LSI model")
    
    # Vẽ biểu đồ 11-point interpolated precision-recall curves
    plt.figure(figsize=(10, 6))
    recall_levels = np.arange(0.0, 1.1, 0.1)
    
    plt.plot(recall_levels, lsi_results['interpolated_precisions'], 'bo-', 
             label='LSI Model', linewidth=2, markersize=6)
    plt.plot(recall_levels, tfidf_results['interpolated_precisions'], 'ro-', 
             label='TF-IDF Baseline', linewidth=2, markersize=6)
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('11-Point Interpolated Precision-Recall Curves')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 1)
    plt.ylim(0, max(max(lsi_results['interpolated_precisions']), 
                   max(tfidf_results['interpolated_precisions'])) + 0.1)
    
    plt.tight_layout()
    plt.show()
    
    # Bảng so sánh chi tiết
    print("\n📋 11-Point Interpolated Precision Values:")
    print("Recall | LSI Model | TF-IDF   | Chênh lệch")
    print("-------|-----------|----------|----------")
    for i, recall in enumerate(recall_levels):
        lsi_prec = lsi_results['interpolated_precisions'][i]
        tfidf_prec = tfidf_results['interpolated_precisions'][i]
        diff = lsi_prec - tfidf_prec
        print(f"{recall:6.1f} | {lsi_prec:8.4f}  | {tfidf_prec:8.4f} | {diff:+8.4f}")
    
    print("\n" + "="*70)
    print("ĐÁNH GIÁ HOÀN THÀNH")
    print("="*70)
    
    # Nhận xét về kết quả
    print(f"\n💡 Nhận xét:")
    print(f"- LSI model sử dụng {lsi_model.n_components} components từ SVD")
    print(f"- Explained variance: {sum(lsi_model.svd.explained_variance_ratio_):.2%}")
    print(f"- LSI có thể tìm ra semantic relationships mà TF-IDF bỏ lỡ")
    print(f"- Trade-off: complexity tăng nhưng có thể cải thiện recall")
    
else:
    print("Không thể thực hiện đánh giá. Vui lòng đảm bảo đã load dữ liệu và train mô hình LSI.")
